In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from scripts.homeos import *
from scripts.plotting import *
from pathlib import Path
import pickle
exp_dir = Path('experiments')
data_dir = exp_dir / 'all_targets'
motif_dir = data_dir / 'motif_fits'

plt.rcParams['xtick.labelsize'] = 14  # font size
plt.rcParams['ytick.labelsize'] = 14
plt.rcParams["font.family"] = "serif"
mpl.rcParams['pdf.fonttype'] = 42  # Use TrueType fonts (editable in Illustrator)
mpl.rcParams['ps.fonttype'] = 42   # Same for EPS
mpl.rcParams['svg.fonttype'] = 'none'  # Keep text as text in SVG
plt.rcParams.update({'font.size': 12, 'text.usetex': True,'text.latex.preamble': r'\usepackage{amsfonts}'})

In [ ]:
#load test error and jacobian norm for models fit to simulated targets 
motif_target_dis_dict = {}
motif_folders = list(motif_dir.glob('*'))
for folder in motif_folders:
    motif_name = folder.name
    #print("Motif: ", motif_name)
    motif_target_dis_dict[motif_name] = {}
    result_files = list(folder.glob('*.npz'))
    for result_file in result_files:
        target_name = result_file.name.split('.')[0][8:]  #skip 'result_'
        print("Target: ", target_name) 
        data = np.load(result_file)
        motif_target_dis_dict[motif_name][target_name] = {
            'train_error': data['training_loss'],
            'test_error': data['test_loss'],
            'jac_fro': data['jac_fro'],
            'jac_spec': data['jac_spec'],
        }

In [ ]:
#load test error and jacobian norm for models fit to compass RNN
avi_motif_dir = Path('experiments') / 'avi' / 'motif_fits' / 'N64_recttanh'
avi_motif_folders = list(avi_motif_dir.glob('*'))
for folder in avi_motif_folders:
    motif_name = folder.name
    result_files = list(folder.glob('*.npz'))
    for result_file in result_files:
        target_name = result_file.name.split('.')[0][8:]  
        print("Target: ", target_name) 
        data = np.load(result_file)
        motif_target_dis_dict[motif_name]['avi'] = {
            'train_error': data['training_loss'],
            'test_error': data['test_loss'],
            'jac_fro': data['jac_fro'],
            'jac_spec': data['jac_spec'],
        }

In [ ]:
#transform data into numpy arrays
n_motifs = len(motif_target_dis_dict.keys())
n_targets = len(motif_target_dis_dict['bla'].keys()) 

# motif_target_train = np.zeros((n_motifs, n_targets))
# for i, motif_name in enumerate(motif_target_dis_dict.keys()):
#     for j, target_name in enumerate(motif_target_dis_dict[motif_name].keys()):
#         motif_target_train[i, j] = motif_target_dis_dict[motif_name][target_name]['train_error']

motif_target_test = np.zeros((n_motifs, n_targets))
for i, motif_name in enumerate(motif_target_dis_dict.keys()):
    for j, target_name in enumerate(motif_target_dis_dict[motif_name].keys()):
        motif_target_test[i, j] = motif_target_dis_dict[motif_name][target_name]['test_error']

motif_target_fro = np.zeros((n_motifs, n_targets))
for i, motif_name in enumerate(motif_target_dis_dict.keys()):
    for j, target_name in enumerate(motif_target_dis_dict[motif_name].keys()):
        motif_target_fro[i, j] = motif_target_dis_dict[motif_name][target_name]['jac_fro']

# motif_target_spec = np.zeros((n_motifs, n_targets))
# for i, motif_name in enumerate(motif_target_dis_dict.keys()):
#     for j, target_name in enumerate(motif_target_dis_dict[motif_name].keys()):

#         motif_target_spec[i, j] = motif_target_dis_dict[motif_name][target_name]['jac_spec']

In [ ]:
motif_target_dis_dict['ring'].keys()

In [ ]:
# name mapping for plotting
target_sns = list(motif_target_dis_dict['bla'].keys())
target_order = ['ra_trajectories_target', 'ra_trajectories_target_nstd0', 'homeo_pert_ring', 'avi',
                 'vdp_trajectories_target', 'noisy_vdp_trajectories_target', 'selkov_trajectories_target', 'lienard_sigmoid_trajectories_target'
                 ] 
target_name_map = {'lienard_sigmoid_trajectories_target': 'Lienard sigmoid',
                   'selkov_trajectories_target': 'Selkov',
                    'vdp_trajectories_target': 'Van der Pol',
                   'noisy_vdp_trajectories_target': 'Noisy Van der Pol',
                     'ra_trajectories_target': 'Ring attractor',
                     'ra_trajectories_target_nstd0': 'Noisy RA',
                     'homeo_pert_ring': 'Homeom. deformed RA',
                     'vf_pert_ring': 'VF deformed RA',
                     'bila_trajs': '2 BLAs',
                        'avi': 'RNN',
}

source_sns = list(motif_target_dis_dict.keys())
source_name_map = {'bla': 'Bounded Line Attractor',
                   'bistable': 'Bistable system',
                   'lds': 'Single fixed point',
                   'lc': 'Limit cycle',
                   'ring': 'Ring attractor',
}

# Specify the desired target order
target_order = [
    'ra_trajectories_target',
    'ra_trajectories_target_nstd0',
    'homeo_pert_ring',
    'vf_pert_ring',
    'avi',
    'vdp_trajectories_target',
    'noisy_vdp_trajectories_target',
    'selkov_trajectories_target',
    'lienard_sigmoid_trajectories_target',
    'bila_trajs'
]

source_order = ['ring', 'lc', 'lds', 'bistable', 'bla']
# Current order in the matrix
current_targets = list(motif_target_dis_dict['ring'].keys())
current_sources = list(motif_target_dis_dict.keys())

# Compute index permutations
target_indices = [current_targets.index(t) for t in target_order]
source_indices = [current_sources.index(s) for s in source_order]

# Permute matrix
motif_target_test_permuted = motif_target_test[np.ix_(source_indices, target_indices)]
motif_target_fro_permuted = motif_target_fro[np.ix_(source_indices, target_indices)]

# Build label mappings (fallback to raw name if not in dict)
xtick_labels = [target_name_map.get(t, t) for t in target_order]
ytick_labels = [source_name_map.get(s, s) for s in source_order]

# Plot
# fig, ax = plt.subplots(figsize=(5, 4))
# hinton(motif_target_test_permuted/motif_target_test_permuted.max(axis=0), ax=ax, max_weight=2)
# ax.set_title("Dissimilarity")

# ax.set_xticks(np.arange(len(target_order)))
# ax.set_yticks(np.arange(len(source_order)))
# ax.set_xticklabels(xtick_labels, rotation=45, ha='right')
# ax.set_yticklabels(ytick_labels)
# plt.tight_layout()
# plt.savefig(data_dir/'motif_target_dissimilarity_row.pdf', dpi=300, bbox_inches='tight')
# plt.show()


In [ ]:
test_rn = motif_target_test_permuted/motif_target_test_permuted.max(axis=0)
comp_rn = motif_target_fro_permuted/motif_target_fro_permuted.max(axis=0)
fig, ax = plt.subplots(figsize=(8, 8))
hinton_double(test_rn, comp_rn, ax=ax, max_weight=1, color1='#ffff00', color2='skyblue', alpha=0.5)
ax.set_xticks(np.arange(len(target_order)))
ax.set_yticks(np.arange(len(source_order)))
ax.set_xticklabels(xtick_labels, rotation=45, ha='right')
ax.set_yticklabels(ytick_labels)
plt.tight_layout()
plt.savefig(data_dir/'motif_target_discomp_row.pdf', dpi=300, bbox_inches='tight')

In [ ]:
test_rn = motif_target_test_permuted/motif_target_test_permuted.max(axis=1).reshape(-1, 1)
comp_rn = motif_target_fro_permuted/motif_target_fro_permuted.max(axis=1).reshape(-1, 1)
fig, ax = plt.subplots(figsize=(8, 8))
hinton_double(test_rn, comp_rn, ax=ax, max_weight=1, color1='#ffff00', color2='skyblue', alpha=0.5)
ax.set_xticks(np.arange(len(target_order)))
ax.set_yticks(np.arange(len(source_order)))
ax.set_xticklabels(xtick_labels, rotation=45, ha='right')
ax.set_yticklabels(ytick_labels)
plt.tight_layout()
plt.savefig(data_dir/'motif_target_discomp_column.pdf', dpi=300, bbox_inches='tight')

# Trajectories and invariant manifolds

In [ ]:
from scripts.time_series import *
fig_dir = Path('figures') / 'all_sources'
def normalize_scale_pair_np(trajectories_target_full, training_pairs=False):
    mean = np.mean(trajectories_target_full, axis=(0, 1), keepdims=True)
    std = np.std(trajectories_target_full, axis=(0, 1), keepdims=True)

    normalized = (trajectories_target_full - mean) / (std + 1e-8)

    if training_pairs:
        return trajectories_target_full, normalized, mean, std
    else:
        return normalized, normalized, mean, std

folder = motif_dir
target_files = list(data_dir.glob('*.npy'))[1:] #ignore avi for now
archtype_folders = list(folder.glob('*'))

print("Targets:")
current_targets = []
for tf in target_files:
    print(tf.name)
    current_targets.append(tf.name[:-4]) #remove .npy
print("\n")
print("Archetypes:")
for af in archtype_folders:
    print(af.name)
target_order = [
    'ra',
    'ra_nstd0.25',
    'homeo_pert_ring',
    'vf_pert_ring',
    'vdp',
    'noisy_vdp',
    'selkov',
    'lienard_sigmoid',
    'bila_trajs'
]
target_indices = [current_targets.index(t) for t in target_order]

target_names = ['Ring attractor', 'Noisy RA', 'deformed RA', 'perturbed RA', 'Van der Pol', 'Noisy VdP', 'Selkov', 'Lienard', '2 BLAs']
target_files = [target_files[i] for i in target_indices]


In [ ]:
#first target
#last archetype (ring)
archtype_j = -1
ring_target_fits = list(archtype_folders[archtype_j].glob('*.npz'))
traj_source_files = list(archtype_folders[archtype_j].glob('traj_motif_source*.npy'))
traj_motif_transformed_files = list(archtype_folders[archtype_j].glob('traj_motif_transformed*.npy'))
ring_target_fits = [ring_target_fits[i] for i in target_indices]
traj_source_files = [traj_source_files[i] for i in target_indices]
traj_motif_transformed_files = [traj_motif_transformed_files[i] for i in target_indices]

fig, axes = plt.subplots(3, 3, figsize=(9, 9))
axes = axes.flatten()  # Flatten 3x3 array into a list of 9 axes

for target_i in range(len(target_files)):
    ax = axes[target_i]

    trajectories_target = np.load(target_files[target_i])
    trajectories_target_full, trajectories_target, mean, std = normalize_scale_pair_np(trajectories_target, False)

    #traj_motif_source = np.load(traj_source_files[target_i])
    traj_motif_transformed = np.load(traj_motif_transformed_files[target_i])
    inv_man = np.load(ring_target_fits[target_i])['inv_man']

    for i in range(trajectories_target.shape[0]):
        ax.plot(trajectories_target[i, :, 0], trajectories_target[i, :, 1], color='darkblue', alpha=0.5, label='target trajectories' if target_i == 7 and i==0 else "")
    for i in range(traj_motif_transformed.shape[0]):
        ax.plot(traj_motif_transformed[i, :, 0], traj_motif_transformed[i, :, 1], color='firebrick', alpha=0.5, label='source trajectories (mapped)' if target_i == 7 and i==0 else "")
    
    ax.plot(inv_man[:, 0], inv_man[:, 1], '--', color='firebrick', label='source invariant manifold' if target_i == 7 else "")
    ax.set_aspect('equal')
    ax.axis('off')
    ax.set_title(target_names[target_i], fontsize=14)

plt.tight_layout()
axes[-2].legend(loc='lower center', fontsize=14, bbox_to_anchor=(0.5, -0.16), ncol=3)
plt.savefig(fig_dir / 'ring_traj_invman.pdf', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
#first archetype (bistable)
archtype_j = 0
target_fits = list(archtype_folders[archtype_j].glob('*.npz'))
traj_source_files = list(archtype_folders[archtype_j].glob('traj_motif_source*.npy'))
traj_motif_transformed_files = list(archtype_folders[archtype_j].glob('traj_motif_transformed*.npy'))
target_fits = [target_fits[i] for i in target_indices]
traj_source_files = [traj_source_files[i] for i in target_indices]
traj_motif_transformed_files = [traj_motif_transformed_files[i] for i in target_indices]

fig, axes = plt.subplots(3, 3, figsize=(9, 9))
axes = axes.flatten()  # Flatten 3x3 array into a list of 9 axes

for target_i in range(len(target_files)):
    ax = axes[target_i]

    trajectories_target = np.load(target_files[target_i])
    trajectories_target_full, trajectories_target, mean, std = normalize_scale_pair_np(trajectories_target, False)

    #traj_motif_source = np.load(traj_source_files[target_i])
    traj_motif_transformed = np.load(traj_motif_transformed_files[target_i])
    inv_man = np.load(target_fits[target_i])['inv_man']

    for i in range(trajectories_target.shape[0]):
        ax.plot(trajectories_target[i, :, 0], trajectories_target[i, :, 1], color='darkblue', alpha=0.5, label='target trajectories' if target_i == 7 and i==0 else "")
    for i in range(traj_motif_transformed.shape[0]):
        ax.plot(traj_motif_transformed[i, :, 0], traj_motif_transformed[i, :, 1], color='firebrick', alpha=0.5, label='source trajectories (mapped)' if target_i == 7 and i==0 else "")
    
    ax.plot(inv_man[:, 0], inv_man[:, 1], '.', color='firebrick', label='source invariant manifold' if target_i == 7 else "")
    ax.set_aspect('equal')
    ax.axis('off')
    ax.set_title(target_names[target_i], fontsize=14)


plt.tight_layout()
axes[-2].legend(loc='lower center', fontsize=14, bbox_to_anchor=(0.5, -0.16), ncol=3)
plt.savefig(fig_dir/'bistable_traj_invman.pdf', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
#second archetype (bla)
archtype_j = 1
target_fits = list(archtype_folders[archtype_j].glob('*.npz'))
traj_source_files = list(archtype_folders[archtype_j].glob('traj_motif_source*.npy'))
traj_motif_transformed_files = list(archtype_folders[archtype_j].glob('traj_motif_transformed*.npy'))
target_fits = [target_fits[i] for i in target_indices]
traj_source_files = [traj_source_files[i] for i in target_indices]
traj_motif_transformed_files = [traj_motif_transformed_files[i] for i in target_indices]

fig, axes = plt.subplots(3, 3, figsize=(9, 9))
axes = axes.flatten()  # Flatten 3x3 array into a list of 9 axes

for target_i in range(len(target_files)):
    ax = axes[target_i]

    trajectories_target = np.load(target_files[target_i])
    trajectories_target_full, trajectories_target, mean, std = normalize_scale_pair_np(trajectories_target, False)

    #traj_motif_source = np.load(traj_source_files[target_i])
    traj_motif_transformed = np.load(traj_motif_transformed_files[target_i])
    inv_man = np.load(target_fits[target_i])['inv_man']

    for i in range(trajectories_target.shape[0]):
        ax.plot(trajectories_target[i, :, 0], trajectories_target[i, :, 1], color='darkblue', alpha=0.5, label='target trajectories' if target_i == 7 and i==0 else "")
    for i in range(traj_motif_transformed.shape[0]):
        ax.plot(traj_motif_transformed[i, :, 0], traj_motif_transformed[i, :, 1], color='firebrick', alpha=0.5, label='source trajectories (mapped)' if target_i == 7 and i==0 else "")
    
    ax.plot(inv_man[:, 0], inv_man[:, 1], '--', color='firebrick', label='source invariant manifold' if target_i == 7 else "")
    ax.set_aspect('equal')
    ax.axis('off')
    ax.set_title(target_names[target_i], fontsize=14)

plt.tight_layout()
axes[-2].legend(loc='lower center', fontsize=14, bbox_to_anchor=(0.5, -0.16), ncol=3)
plt.savefig(fig_dir/'bla_traj_invman.pdf', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
#third archetype (lc)
archtype_j = 2
target_fits = list(archtype_folders[archtype_j].glob('*.npz'))
traj_source_files = list(archtype_folders[archtype_j].glob('traj_motif_source*.npy'))
traj_motif_transformed_files = list(archtype_folders[archtype_j].glob('traj_motif_transformed*.npy'))
target_fits = [target_fits[i] for i in target_indices]
traj_source_files = [traj_source_files[i] for i in target_indices]
traj_motif_transformed_files = [traj_motif_transformed_files[i] for i in target_indices]

fig, axes = plt.subplots(3, 3, figsize=(9, 9))
axes = axes.flatten()  # Flatten 3x3 array into a list of 9 axes

for target_i in range(len(target_files)):
    ax = axes[target_i]

    trajectories_target = np.load(target_files[target_i])
    trajectories_target_full, trajectories_target, mean, std = normalize_scale_pair_np(trajectories_target, False)

    #traj_motif_source = np.load(traj_source_files[target_i])
    traj_motif_transformed = np.load(traj_motif_transformed_files[target_i])
    inv_man = np.load(target_fits[target_i])['inv_man']

    for i in range(trajectories_target.shape[0]):
        ax.plot(trajectories_target[i, :, 0], trajectories_target[i, :, 1], color='darkblue', alpha=0.5, label='target trajectories' if target_i == 7 and i==0 else "")
    for i in range(traj_motif_transformed.shape[0]):
        ax.plot(traj_motif_transformed[i, :, 0], traj_motif_transformed[i, :, 1], color='firebrick', alpha=0.5, label='source trajectories (mapped)' if target_i == 7 and i==0 else "")
    
    ax.plot(inv_man[:, 0], inv_man[:, 1], '--', color='firebrick', label='source invariant manifold' if target_i == 7 else "")
    ax.set_aspect('equal')
    ax.axis('off')
    ax.set_title(target_names[target_i], fontsize=14)

plt.tight_layout()
axes[-2].legend(loc='lower center', fontsize=14, bbox_to_anchor=(0.5, -0.16), ncol=3)
plt.savefig(fig_dir/'lc_traj_invman.pdf', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
#fourth archetype (lds)
archtype_j = 3
target_fits = list(archtype_folders[archtype_j].glob('*.npz'))
traj_source_files = list(archtype_folders[archtype_j].glob('traj_motif_source*.npy'))
traj_motif_transformed_files = list(archtype_folders[archtype_j].glob('traj_motif_transformed*.npy'))
target_fits = [target_fits[i] for i in target_indices]
traj_source_files = [traj_source_files[i] for i in target_indices]
traj_motif_transformed_files = [traj_motif_transformed_files[i] for i in target_indices]

fig, axes = plt.subplots(3, 3, figsize=(9, 9))
axes = axes.flatten()  # Flatten 3x3 array into a list of 9 axes

for target_i in range(len(target_files)):
    ax = axes[target_i]

    trajectories_target = np.load(target_files[target_i])
    trajectories_target_full, trajectories_target, mean, std = normalize_scale_pair_np(trajectories_target, False)

    #traj_motif_source = np.load(traj_source_files[target_i])
    traj_motif_transformed = np.load(traj_motif_transformed_files[target_i])
    inv_man = np.load(target_fits[target_i])['inv_man']

    for i in range(trajectories_target.shape[0]):
        ax.plot(trajectories_target[i, :, 0], trajectories_target[i, :, 1], color='darkblue', alpha=0.5, label='target trajectories' if target_i == 7 and i==0 else "")
    for i in range(traj_motif_transformed.shape[0]):
        ax.plot(traj_motif_transformed[i, :, 0], traj_motif_transformed[i, :, 1], color='firebrick', alpha=0.5, label='source trajectories (mapped)' if target_i == 7 and i==0 else "")
    
    
    ax.scatter(inv_man[0], inv_man[1], color='firebrick', label='source invariant manifold' if target_i == 7 else "")
    ax.set_aspect('equal')
    ax.axis('off')
    ax.set_title(target_names[target_i], fontsize=14)

plt.tight_layout()
axes[-2].legend(loc='lower center', fontsize=14, bbox_to_anchor=(0.5, -0.16), ncol=3)
plt.savefig(fig_dir/'lds_traj_invman.pdf', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# folder = motif_dir / 'lc' 
# result_files = list(folder.glob('parameters*'))

# folder = motif_dir / 'lc' 
# hdsn_files = list(folder.glob('*.pth'))
# hdsn_files, result_files
# for fi, file in enumerate(result_files[:]):
#     if fi == 0:
#         continue
#     print(fi,file)
#     all_parameters = pickle.load(open(file, "rb"))
#     homeo = build_homeomorphism(all_parameters['homeo_params'])
#     dynsys = build_ds_motif(**all_parameters['ds_params'])
#     print(all_parameters['ds_params']['dt'])
#     # dynsys.alpha = -1. 
#     homeo_ds_net = load_homeo_ds_net(hdsn_files[fi], homeo, dynsys)
#     print(hdsn_files[fi].name, "v =", np.round(homeo_ds_net.dynamical_system.velocity.item()/dynsys.dt,3))